# Imports

In [ ]:
%run Imports.ipynb
name = 'Kred'

# specific imports
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV


key_featsubgroups = pd.read_pickle('../pickle/2_FS/' + name + '/key_featsubgroups.pkl')
df = pd.read_pickle('../pickle/2_FS/' + name + '/2_df_new_.pkl')

2025-06-05 15:54:58.578859: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-05 15:54:58.580551: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-05 15:54:58.584232: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-05 15:54:58.592297: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749131698.609763 3464668 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749131698.61

In [4]:
print(df[target].value_counts()/df.shape[0])
print('df_shape: ', df.shape)

arrears
1   0.646
0   0.354
Name: count, dtype: float64
df_shape:  (129457, 418)


In [ ]:
# param grid scikit 
param_grid = {
    'hidden_layer_sizes': [(h,) for h in [5, 10, 20, 50, 100, 200, 300, 406]],
    'activation': ['relu', 'tanh', 'logistic'],                   # activation functions
    'solver': ['adam', 'sgd'],                                    # optimization algorithms
    'alpha': [0.0001, 0.001, 0.01, 0.1],                                # L2 regularization
    'learning_rate': ['constant', 'adaptive'],                    # learning rate strategy
    'learning_rate_init': [0.001, 0.005, 0.01, 0.05],                           # initial learning rate
    'max_iter': [100, 200, 500, 750, 1000]                                        # training epochs
}

# 2) Create Model prediction functions

## 2.1) Split dataset into train/testing while excluding demographic features

In [5]:
def split_data_4(df, key_featsubgroups=key_featsubgroups, target=target, test_size=0.2, random_state=42):
    """
    Splits the dataset into training and testing sets while excluding demographic features.

    Parameters:
    df (DataFrame): The dataset containing features and target variable.
    key_featsubgroups (DataFrame): A mapping of feature subgroups.
    target (str): The name of the target variable.
    test_size (float, optional): The proportion of the dataset to allocate for testing. Default is 0.2.
    random_state (int, optional): Random seed for reproducibility. Default is 42.

    Returns:
    tuple: X_train, X_test, y_train, y_test (training and testing datasets)
    """

    # Extract demographic features
    demo_feat = key_featsubgroups.loc[key_featsubgroups['subgroup'] == 'demo', 'list_features'].values[0]
    print("Demographic Features:", demo_feat)

    # Separate features (X) and target variable (y), excluding demographic features
    X = df.drop(columns=[target] + demo_feat)
    y = df[target]

    # Split the dataset into training (80%) and testing (20%) sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    # Print dataset shapes
    print(f"Training Features Shape: {X_train.shape}")
    print(f"Training Labels Shape: {y_train.shape}")
    print(f"Testing Features Shape: {X_test.shape}")
    print(f"Testing Labels Shape: {y_test.shape}")

    return X_train, X_test, y_train, y_test

## 2.2) Model training and prediction

In [6]:
x_train, x_test, y_train, y_test = split_data_4(df)

Demographic Features: ['clientdata.demo.gender', 'clientdata.demo.age_year', 'clientdata.demo.age_month', 'clientdata.demo.children', 'clientdata.demo.children_singleparent', 'clientdata.demo.maritalstatus_expand_SINGLE', 'clientdata.demo.maritalstatus_expand_MARRIED', 'clientdata.demo.maritalstatus_expand_DIVORCED', 'clientdata.demo.maritalstatus_expand_WIDOWED', 'clientdata.demo.maritalstatus_expand_newvalue', 'clientdata.demo.maritalstatus_woe']
Training Features Shape: (103565, 406)
Training Labels Shape: (103565,)
Testing Features Shape: (25892, 406)
Testing Labels Shape: (25892,)


## 7) Neural Network Sklearn

In [ ]:
def sklearn_nn(default_params, param_grid, df, results_dict):
    """
    Performs Bayesian hyperparameter optimization for an MLP classifier with one hidden layer using Optuna.

    Parameters:
    -----------
    default_params : dict
        Default hyperparameters (each as a list).
    param_grid : dict
        Search space for hyperparameters.
    df : DataFrame
        Input data.
    results_dict : dict
        Dictionary for storing evaluation results.

    Returns:
    --------
    best_params : dict
        Best-found parameters by Optuna.
    results_dict : dict
        Updated dictionary with evaluation metrics.
    """

    # You must define or import split_data_4 and model_pred
    X_train, X_test, y_train, y_test = split_data_4(df)

    n_trials = int(np.cumsum([len(v) for v in param_grid.values()])[-1])
    print("No. of trials:", n_trials)

    def objective(trial):
        params = {key: trial.suggest_categorical(key, values) for key, values in param_grid.items()}
        clf = MLPClassifier(**params)
        return cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy').mean()

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)

    best_params = study.best_params
    clf = MLPClassifier(**best_params)

    results_dict = model_pred(X_train, X_test, y_train, y_test, clf, 'mlp_optuna', 'opt', results_dict)

    print(f"Optimization completed in {(time.time() - start_time) / 60:.2f} minutes")
    return best_params, results_dict
